In [8]:
from scipy.io import loadmat
import pandas as pd

# Load the .mat file
data = loadmat('pwr/PWR_exp_002.mat')

pwr = data['PWR']

In [29]:
# Flattening to extract column list
columns = [item[0] for item in pwr[0]]

# Making a DataFrame for readability
df = pd.DataFrame(data=pwr[1:], columns=columns)

In [30]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [31]:
# Show raw value from the first row of the timestamp column
print(repr(df['timestamp'].iloc[0]))


array([[array(['15:07:30.646'], dtype='<U12')]], dtype=object)


In [35]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Step 1: Flatten the nested structure AND cast to native Python string
df['timestamp'] = df['timestamp'].apply(lambda x: str(x[0][0][0]) if isinstance(x, np.ndarray) else str(x))

# Step 2: Parse the string as datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%H:%M:%S.%f", errors='coerce')

# Step 3: Check
print(df['timestamp'].head())

0   1900-01-01 15:07:30.646
1   1900-01-01 15:07:30.764
2   1900-01-01 15:07:30.847
3   1900-01-01 15:07:30.948
4   1900-01-01 15:07:31.047
Name: timestamp, dtype: datetime64[ns]


In [41]:
from datetime import timedelta

window_size = timedelta(seconds=3)
row_counts = []

start_idx = 0
while start_idx < len(df):
    start_time = df.iloc[start_idx]['timestamp']
    end_time = start_time + window_size

    window = df[(df['timestamp'] >= start_time) & (df['timestamp'] < end_time)]

    if len(window) == 0:
        start_idx += 1
        continue

    row_counts.append(len(window))

    # Move to the next non-overlapping window
    start_idx = window.index[-1] + 1

# Final average
avg_rows = sum(row_counts) / len(row_counts)

print(f"Average rows per 3-second window: {avg_rows:.2f}")


Average rows per 3-second window: 30.16
